In [4]:
# This is my main readiness code. 
# Prompts you to select folder and adds info into sqlite database at Readiness_Screen_Data.db

import os
import sqlite3
import tkinter as tk
from tkinter import filedialog
import xml.etree.ElementTree as ET
import pandas as pd

# Database file path
db_path = 'D:/Readiness Screen 3/Readiness_Screen_Data.db'

# Establish connection and create tables
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# -------------------------------------------------
# 1. Create/Update Tables with Correct Column Order
# -------------------------------------------------
cursor.executescript("""
CREATE TABLE IF NOT EXISTS Participant (
    Participant_ID INTEGER PRIMARY KEY AUTOINCREMENT,
    Name TEXT,
    Height REAL,
    Weight REAL,
    Plyo_Day TEXT,
    Creation_Date TEXT
);

CREATE TABLE IF NOT EXISTS I (
    Trial_ID INTEGER PRIMARY KEY AUTOINCREMENT,
    Name TEXT,
    Participant_ID INTEGER,
    Avg_Force REAL,
    Avg_Force_Norm REAL,
    Max_Force REAL,
    Max_Force_Norm REAL,
    Time_to_Max REAL,
    Creation_Date TEXT,
    FOREIGN KEY (Participant_ID) REFERENCES Participant(Participant_ID)
);

CREATE TABLE IF NOT EXISTS Y (
    Trial_ID INTEGER PRIMARY KEY AUTOINCREMENT,
    Name TEXT,
    Participant_ID INTEGER,
    Avg_Force REAL,
    Avg_Force_Norm REAL,
    Max_Force REAL,
    Max_Force_Norm REAL,
    Time_to_Max REAL,
    Creation_Date TEXT,
    FOREIGN KEY (Participant_ID) REFERENCES Participant(Participant_ID)
);

CREATE TABLE IF NOT EXISTS T (
    Trial_ID INTEGER PRIMARY KEY AUTOINCREMENT,
    Name TEXT,
    Participant_ID INTEGER,
    Avg_Force REAL,
    Avg_Force_Norm REAL,
    Max_Force REAL,
    Max_Force_Norm REAL,
    Time_to_Max REAL,
    Creation_Date TEXT,
    FOREIGN KEY (Participant_ID) REFERENCES Participant(Participant_ID)
);

CREATE TABLE IF NOT EXISTS IR90 (
    Trial_ID INTEGER PRIMARY KEY AUTOINCREMENT,
    Name TEXT,
    Participant_ID INTEGER,
    Avg_Force REAL,
    Avg_Force_Norm REAL,
    Max_Force REAL,
    Max_Force_Norm REAL,
    Time_to_Max REAL,
    Creation_Date TEXT,
    FOREIGN KEY (Participant_ID) REFERENCES Participant(Participant_ID)
);

CREATE TABLE IF NOT EXISTS CMJ (
    Trial_ID INTEGER PRIMARY KEY AUTOINCREMENT,
    Name TEXT,
    Participant_ID INTEGER,
    Jump_Height REAL,
    Peak_Power REAL,
    Peak_Force REAL,
    Creation_Date TEXT,
    FOREIGN KEY (Participant_ID) REFERENCES Participant(Participant_ID)
);

CREATE TABLE IF NOT EXISTS PPU (
    Trial_ID INTEGER PRIMARY KEY AUTOINCREMENT,
    Name TEXT,
    Participant_ID INTEGER,
    Jump_Height REAL,
    Peak_Power REAL,
    Peak_Force REAL,
    Creation_Date TEXT,
    FOREIGN KEY (Participant_ID) REFERENCES Participant(Participant_ID)
);
""")
conn.commit()

# -------------------------------------------------
# 2. Prompt user to select a folder
# -------------------------------------------------
root = tk.Tk()
root.withdraw()
selected_folder = filedialog.askdirectory(initialdir='D:/Readiness Screen 3/Data/')

if not selected_folder:
    print("No folder selected. Exiting...")
    exit()

# -------------------------------------------------
# 3. Locate the XML file (assuming 'sessionXYZ.xml')
# -------------------------------------------------
xml_file_path = ''
for root_dir, _, files in os.walk(selected_folder):
    for file in files:
        if file.lower().startswith('session') and file.lower().endswith('.xml'):
            xml_file_path = os.path.join(root_dir, file)
            break
    if xml_file_path:
        break

if not xml_file_path:
    print("No XML file found. Exiting...")
    exit()

# -------------------------------------------------
# 4. Parse the XML file
# -------------------------------------------------
tree = ET.parse(xml_file_path)
xml_root = tree.getroot()

def find_text(element, tag):
    found = element.find(tag)
    return found.text if found is not None else None

session_fields = xml_root.find(".//Session/Fields")
name = find_text(session_fields, "Name")
height = find_text(session_fields, "Height")
weight = find_text(session_fields, "Weight")
plyo_day = find_text(session_fields, "Plyo_Day")
creation_date = find_text(session_fields, "Creation_date")

if None in [name, height, weight, plyo_day, creation_date]:
    print("Missing data in XML file. Exiting...")
    exit()

# -------------------------------------------------
# 5. Insert participant data
# -------------------------------------------------
cursor.execute("""
INSERT INTO Participant (Name, Height, Weight, Plyo_Day, Creation_Date)
VALUES (?, ?, ?, ?, ?)
""", (name, height, weight, plyo_day, creation_date))

participant_id = cursor.lastrowid
conn.commit()

# -------------------------------------------------
# 6. Define ASCII file mapping and output path
# -------------------------------------------------
ascii_files = {
    "I": "i_data.txt",
    "Y": "y_data.txt",
    "T": "t_data.txt",
    "IR90": "ir90_data.txt",
    "CMJ": "cmj_data.txt",
    "PPU": "ppu_data.txt"
}
output_path = 'D:/Readiness Screen 3/Output Files/'

# -------------------------------------------------
# 7. Process files in the ASCII file dictionary
# -------------------------------------------------
for file_key, file_name in ascii_files.items():
    file_path = os.path.join(output_path, file_name)
    
    if os.path.exists(file_path):
        print(f"Processing file: {file_path}")
        
        try:
            # Depending on table, parse the correct columns 
            # WITHOUT reading creation_date from the .txt
            if file_key in ["I", "Y", "T", "IR90"]:
                # The text file columns: Max_Force, Max_Force_Norm, Avg_Force, Avg_Force_Norm, Time_to_Max
                headers = ["Max_Force", "Max_Force_Norm", "Avg_Force", "Avg_Force_Norm", "Time_to_Max"]
                df = pd.read_csv(file_path, sep='\s+', skiprows=5, names=headers)

            elif file_key in ["CMJ", "PPU"]:
                # The text file columns: JH_IN, LEWIS_PEAK_POWER, Max_Force
                headers = ["JH_IN", "LEWIS_PEAK_POWER", "Max_Force"]
                df = pd.read_csv(file_path, sep='\s+', skiprows=5, names=headers)

            # Print a preview for debugging
            print(f"Data from {file_name} (first 5 rows):")
            print(df.head())

            # Insert data into the database
            table_name = file_key.upper()  # e.g., "I", "Y", etc.
            
            # Insert each row, attaching the same XML creation_date
            for _, row in df.iterrows():
                if file_key in ["I", "Y", "T", "IR90"]:
                    cursor.execute(f"""
                        INSERT INTO {table_name}
                        (Name, Participant_ID, Avg_Force, Avg_Force_Norm, Max_Force, Max_Force_Norm, Time_to_Max, Creation_Date)
                        VALUES (?, ?, ?, ?, ?, ?, ?, ?)
                    """, (
                        name,
                        participant_id,
                        row['Avg_Force'],
                        row['Avg_Force_Norm'],
                        row['Max_Force'],
                        row['Max_Force_Norm'],
                        row['Time_to_Max'],
                        creation_date    # from the XML
                    ))

                elif file_key in ["CMJ", "PPU"]:
                    cursor.execute(f"""
                        INSERT INTO {table_name}
                        (Name, Participant_ID, Jump_Height, Peak_Power, Peak_Force, Creation_Date)
                        VALUES (?, ?, ?, ?, ?, ?)
                    """, (
                        name,
                        participant_id,
                        row['JH_IN'],
                        row['LEWIS_PEAK_POWER'],
                        row['Max_Force'],
                        creation_date    # from the XML
                    ))
                    
        except Exception as e:
            print(f"Error processing {file_name}: {e}")
            
    else:
        print(f"File not found: {file_path}")

# -------------------------------------------------
# 8. Final Commit and Close
# -------------------------------------------------
conn.commit()
conn.close()

print("Data successfully added to the database.")


Processing file: D:/Readiness Screen 3/Output Files/i_data.txt
Data from i_data.txt (first 5 rows):
   Max_Force  Max_Force_Norm  Avg_Force  Avg_Force_Norm  Time_to_Max
1      158.2           158.2      124.6          124.56         1.65
Processing file: D:/Readiness Screen 3/Output Files/y_data.txt
Data from y_data.txt (first 5 rows):
   Max_Force  Max_Force_Norm  Avg_Force  Avg_Force_Norm  Time_to_Max
1      131.5           131.5      111.3          111.32         3.02
Processing file: D:/Readiness Screen 3/Output Files/t_data.txt
Data from t_data.txt (first 5 rows):
   Max_Force  Max_Force_Norm  Avg_Force  Avg_Force_Norm  Time_to_Max
1      113.9           113.9       96.3           96.32         2.05
Processing file: D:/Readiness Screen 3/Output Files/ir90_data.txt
Data from ir90_data.txt (first 5 rows):
   Max_Force  Max_Force_Norm  Avg_Force  Avg_Force_Norm  Time_to_Max
1      152.8           152.8      127.5          127.48         2.45
Processing file: D:/Readiness Screen 3/Out

In [5]:
# Reorders the database to be in alphabetical order

import sqlite3


db_path = "D:/Readiness Screen 3/Readiness_Screen_Data.db" 
sort_column = "Name"     

def reorder_all_tables(db_path, sort_column):
    try:
        # Connect to the database
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()
        
        # Fetch all table names in the database
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
        tables = cursor.fetchall()

        for table in tables:
            table_name = table[0]

            # Skip system tables like sqlite_sequence
            if table_name.startswith("sqlite_"):
                continue

            print(f"Processing table: {table_name}")

            # Check if the column exists in the current table
            cursor.execute(f"PRAGMA table_info({table_name});")
            columns = [info[1] for info in cursor.fetchall()]
            if sort_column not in columns:
                print(f"Skipping table '{table_name}' - Column '{sort_column}' not found.")
                continue

            # Create a new sorted table
            temp_table = f"{table_name}_sorted"
            cursor.execute(f"CREATE TABLE {temp_table} AS SELECT * FROM {table_name} ORDER BY {sort_column} ASC;")
            
            # Drop the old table
            cursor.execute(f"DROP TABLE {table_name};")
            
            # Rename the new table to the original name
            cursor.execute(f"ALTER TABLE {temp_table} RENAME TO {table_name};")
            print(f"Table '{table_name}' reordered successfully.")

        # Commit changes
        conn.commit()
        print("All tables processed.")
    except sqlite3.Error as e:
        print(f"An error occurred: {e}")
    finally:
        conn.close()

reorder_all_tables(db_path, sort_column)


Processing table: Participant
Table 'Participant' reordered successfully.
Processing table: I
Table 'I' reordered successfully.
Processing table: Y
Table 'Y' reordered successfully.
Processing table: T
Table 'T' reordered successfully.
Processing table: IR90
Table 'IR90' reordered successfully.
Processing table: CMJ
Table 'CMJ' reordered successfully.
Processing table: PPU
Table 'PPU' reordered successfully.
All tables processed.


In [6]:
# Creates Dash Report

import sqlite3
import pandas as pd
from dash import Dash, dcc, html, Input, Output
import plotly.graph_objects as go

# ---------------------------
# Step 1: Data Loading
# ---------------------------

conn = sqlite3.connect('D:/Readiness Screen 3/Readiness_Screen_Data.db')

df_cmj = pd.read_sql_query("SELECT Name, Creation_Date, Jump_Height AS Jump_Height_CMJ FROM CMJ;", conn)
df_ppu = pd.read_sql_query("SELECT Name, Creation_Date, Jump_Height AS Jump_Height_PPU FROM PPU;", conn)
df_i = pd.read_sql_query("SELECT Name, Creation_Date, Avg_Force AS Avg_Force_I FROM I;", conn)
df_y = pd.read_sql_query("SELECT Name, Creation_Date, Avg_Force AS Avg_Force_Y FROM Y;", conn)
df_t = pd.read_sql_query("SELECT Name, Creation_Date, Avg_Force AS Avg_Force_T FROM T;", conn)
df_ir90 = pd.read_sql_query("SELECT Name, Creation_Date, Avg_Force AS Avg_Force_IR90 FROM IR90;", conn)
conn.close()

df_merged = df_cmj.merge(df_ppu, on=["Name", "Creation_Date"], how="outer")
df_merged = df_merged.merge(df_i, on=["Name", "Creation_Date"], how="outer")
df_merged = df_merged.merge(df_y, on=["Name", "Creation_Date"], how="outer")
df_merged = df_merged.merge(df_t, on=["Name", "Creation_Date"], how="outer")
df_merged = df_merged.merge(df_ir90, on=["Name", "Creation_Date"], how="outer")

df_merged['Creation_Date'] = pd.to_datetime(df_merged['Creation_Date'])

df_merged = df_merged.sort_values(by="Creation_Date")

participants = df_merged['Name'].dropna().unique()

# ---------------------------
# Step 2: Initialize the Dash App
# ---------------------------

app = Dash(__name__)

# ---------------------------
# Step 3: Dash Layout
# ---------------------------

app.layout = html.Div([
    html.H1("Participant Measurements Over Time"),

    html.Div([
        html.Label("Select a Participant:"),
        dcc.Dropdown(
            id='participant-dropdown',
            options=[{'label': p, 'value': p} for p in participants],
            value=participants[0] if len(participants) > 0 else None,
            clearable=False
        )
    ], style={'width': '30%', 'display': 'inline-block'}),

    dcc.Graph(id='measurements-graph'),
    dcc.Graph(id='jump-heights-graph')  # New plot for CMJ and PPU
])

# ---------------------------
# Step 4: Callbacks
# ---------------------------

@app.callback(
    [Output('measurements-graph', 'figure'),
     Output('jump-heights-graph', 'figure')],  # Updated callback for two plots
    [Input('participant-dropdown', 'value')]
)
def update_graph(selected_participant):
    dff = df_merged[df_merged['Name'] == selected_participant]

    # First plot: Measurements over time
    fig_measurements = go.Figure()
    
    if 'Avg_Force_I' in dff.columns and dff['Avg_Force_I'].notnull().any():
        fig_measurements.add_trace(go.Scatter(
            x=dff['Creation_Date'], y=dff['Avg_Force_I'], mode='lines+markers', name='Avg_Force_I'
        ))

    if 'Avg_Force_Y' in dff.columns and dff['Avg_Force_Y'].notnull().any():
        fig_measurements.add_trace(go.Scatter(
            x=dff['Creation_Date'], y=dff['Avg_Force_Y'], mode='lines+markers', name='Avg_Force_Y'
        ))

    if 'Avg_Force_T' in dff.columns and dff['Avg_Force_T'].notnull().any():
        fig_measurements.add_trace(go.Scatter(
            x=dff['Creation_Date'], y=dff['Avg_Force_T'], mode='lines+markers', name='Avg_Force_T'
        ))

    if 'Avg_Force_IR90' in dff.columns and dff['Avg_Force_IR90'].notnull().any():
        fig_measurements.add_trace(go.Scatter(
            x=dff['Creation_Date'], y=dff['Avg_Force_IR90'], mode='lines+markers', name='Avg_Force_IR90'
        ))

    fig_measurements.update_layout(
        title=f"Measurements Over Time for {selected_participant}",
        xaxis_title="Date",
        yaxis_title="Measurement Value",
        hovermode='x unified'
    )

    # Second plot: Jump Heights (CMJ and PPU)
    fig_jump_heights = go.Figure()

    if 'Jump_Height_CMJ' in dff.columns and dff['Jump_Height_CMJ'].notnull().any():
        fig_jump_heights.add_trace(go.Scatter(
            x=dff['Creation_Date'], y=dff['Jump_Height_CMJ'], mode='lines+markers', name='Jump_Height_CMJ'
        ))

    if 'Jump_Height_PPU' in dff.columns and dff['Jump_Height_PPU'].notnull().any():
        fig_jump_heights.add_trace(go.Scatter(
            x=dff['Creation_Date'], y=dff['Jump_Height_PPU'], mode='lines+markers', name='Jump_Height_PPU'
        ))

    fig_jump_heights.update_layout(
        title=f"Jump Heights Over Time for {selected_participant}",
        xaxis_title="Date",
        yaxis_title="Jump Height (cm)",
        hovermode='x unified'
    )

    return fig_measurements, fig_jump_heights

# ---------------------------
# Step 5: Run the App
# ---------------------------
if __name__ == '__main__':
    app.run_server(debug=True)
